# EDA of Flight Data and Weather Data #

In [19]:
import pandas as pd
from sqlalchemy import create_engine, types
from sqlalchemy import text 
from dotenv import dotenv_values
import plotly.express as px
import plotly.graph_objects as go

In [20]:
config = dotenv_values()

pg_user = config['POSTGRES_USER'] 
pg_host = config['POSTGRES_HOST']
pg_port = config['POSTGRES_PORT']
pg_db = config['POSTGRES_DB']
pg_schema = config['POSTGRES_SCHEMA']
pg_pass = config['POSTGRES_PASS']

url = f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}'

In [21]:
engine = create_engine(url, echo=False)

In [22]:
with engine.begin() as conn: 
    result = conn.execute(text(f'SET search_path TO {pg_schema};'))

flights = pd.read_sql(sql=text("SELECT * FROM debby_prep_flights;"), con=engine)

flights.head(3)

## Get Weather Data ##

In [23]:
weather = pd.read_sql(sql=text("SELECT * FROM debby_prep_weather_daily;"), con=engine)

In [24]:
weather.head(3)

,airport_code,station_id,date,avg_temp_c,min_temp_c,max_temp_c,precipitation_mm,max_snow_mm,avg_wind_direction,avg_wind_speed_kmh,wind_peakgust_kmh,avg_pressure_hpa,sun_minutes,date_day
0,MCO,72205,2024-07-01,27.6,23.3,33.3,0.0,0,174,10.1,None,1015.5,None,1.0
1,TPA,72211,2024-07-01,27.7,25.0,32.2,0.0,0,193,11.9,None,1015.6,None,1.0
2,JFK,74486,2024-07-01,23.3,17.8,27.8,0.0,0,352,24.8,None,1016.2,None,1.0


In [81]:
weather_sum = weather.drop(['airport_code', 'station_id'], axis=1).groupby(['date']).mean().round(2)

In [89]:
weather_sum = weather_sum.reset_index()

In [25]:
weather_TPA = weather.query('airport_code == "TPA"')
weather_TPA.head(3)

,airport_code,station_id,date,avg_temp_c,min_temp_c,max_temp_c,precipitation_mm,max_snow_mm,avg_wind_direction,avg_wind_speed_kmh,wind_peakgust_kmh,avg_pressure_hpa,sun_minutes,date_day
1,TPA,72211,2024-07-01,27.7,25.0,32.2,0.0,0,193,11.9,None,1015.6,None,1.0
4,TPA,72211,2024-07-02,29.4,26.7,33.3,0.0,0,145,9.4,None,1016.7,None,2.0
6,TPA,72211,2024-07-03,29.7,25.0,34.4,6.9,0,76,8.3,None,1017.9,None,3.0


In [26]:
weather_JFK = weather.query('airport_code == "JFK"')
weather_MCO = weather.query('airport_code == "MCO"')

In [27]:
weather_JFK.describe()

,station_id,avg_temp_c,min_temp_c,max_temp_c,precipitation_mm,max_snow_mm,avg_wind_direction,avg_wind_speed_kmh,avg_pressure_hpa,date_day
count,62.0,62.000000,62.000000,62.000000,62.000000,62.0,62.000000,62.000000,62.000000,62.000000
mean,74486.0,24.611290,20.983871,28.874194,2.670968,0.0,194.516129,15.869355,1016.209677,16.000000
std,0.0,1.894003,2.356936,2.335958,6.184019,0.0,80.396098,4.269049,4.175169,9.017288
min,74486.0,19.400000,14.400000,22.800000,0.000000,0.0,3.000000,9.400000,1005.700000,1.000000
25%,74486.0,23.600000,19.550000,27.800000,0.000000,0.0,156.750000,13.000000,1013.375000,8.250000
50%,74486.0,24.800000,21.700000,28.900000,0.000000,0.0,194.000000,15.100000,1016.150000,16.000000
75%,74486.0,25.700000,22.200000,30.450000,1.300000,0.0,235.250000,16.900000,1019.075000,23.750000
max,74486.0,28.400000,25.600000,35.000000,32.500000,0.0,352.000000,31.700000,1024.900000,31.000000


## Get Flight Data ##

In [28]:
flights_information_debby = pd.read_sql(sql=text("SELECT * FROM flights_information_debby;"), con=engine)

In [29]:
flights_MCO = pd.read_sql(sql=text("SELECT * FROM flights_MCO;"), con=engine)
flights_JFK = pd.read_sql(sql=text("SELECT * FROM flights_JFK;"), con=engine)
flights_TPA = pd.read_sql(sql=text("SELECT * FROM flights_TPA;"), con=engine)

In [30]:
flights_information_debby.head(3)

,flight_date,total_flights,total_cancelled,avg_dep_delay,avg_arr_delay
0,2024-07-01,1930,43,21.46,13.07
1,2024-07-02,1907,6,14.85,4.30
2,2024-07-03,1922,2,8.94,-1.75


In [ ]:
flights_MCO = flights_MCO.merge(weather_MCO[['date', 'avg_temp_c', 'precipitation_mm', 'avg_wind_speed_kmh', 'avg_pressure_hpa']], 
                                how='left', 
                                left_on='flight_date', 
                                right_on='date').drop(columns = ['date'])

flights_TPA = flights_TPA.merge(weather_TPA[['date', 'avg_temp_c', 'precipitation_mm', 'avg_wind_speed_kmh', 'avg_pressure_hpa']], 
                                how='left', 
                                left_on='flight_date', 
                                right_on='date').drop(columns = ['date'])

flights_JFK = flights_JFK.merge(weather_JFK[['date', 'avg_temp_c', 'precipitation_mm', 'avg_wind_speed_kmh', 'avg_pressure_hpa']], 
                                how='left', 
                                left_on='flight_date', 
                                right_on='date').drop(columns = ['date'])

In [47]:
flights_MCO_dep = pd.read_sql(sql=text("SELECT * FROM mco_departure;"), con=engine)
flights_MCO_arr = pd.read_sql(sql=text("SELECT * FROM mco_arr;"), con=engine)

flights_TPA_dep = pd.read_sql(sql=text("SELECT * FROM tpa_departure;"), con=engine)
flights_TPA_arr = pd.read_sql(sql=text("SELECT * FROM tpa_arr;"), con=engine)

flights_JFK_dep = pd.read_sql(sql=text("SELECT * FROM jfk_departure;"), con=engine)
flights_JFK_arr = pd.read_sql(sql=text("SELECT * FROM jfk_arr;"), con=engine)

In [48]:
flights_MCO_dep = flights_MCO_dep.merge(weather_MCO[['date', 'avg_temp_c', 'precipitation_mm', 'avg_wind_speed_kmh', 'avg_pressure_hpa']], 
                                how='left', 
                                left_on='flight_date', 
                                right_on='date').drop(columns = ['date'])
flights_MCO_arr = flights_MCO_arr.merge(weather_MCO[['date', 'avg_temp_c', 'precipitation_mm', 'avg_wind_speed_kmh', 'avg_pressure_hpa']], 
                                how='left', 
                                left_on='flight_date', 
                                right_on='date').drop(columns = ['date'])

flights_TPA_dep = flights_TPA_dep.merge(weather_TPA[['date', 'avg_temp_c', 'precipitation_mm', 'avg_wind_speed_kmh', 'avg_pressure_hpa']], 
                                how='left', 
                                left_on='flight_date', 
                                right_on='date').drop(columns = ['date'])
flights_TPA_arr = flights_TPA_arr.merge(weather_TPA[['date', 'avg_temp_c', 'precipitation_mm', 'avg_wind_speed_kmh', 'avg_pressure_hpa']], 
                                how='left', 
                                left_on='flight_date', 
                                right_on='date').drop(columns = ['date'])

flights_JFK_dep = flights_JFK_dep.merge(weather_JFK[['date', 'avg_temp_c', 'precipitation_mm', 'avg_wind_speed_kmh', 'avg_pressure_hpa']], 
                                how='left', 
                                left_on='flight_date', 
                                right_on='date').drop(columns = ['date'])
flights_JFK_arr = flights_JFK_arr.merge(weather_JFK[['date', 'avg_temp_c', 'precipitation_mm', 'avg_wind_speed_kmh', 'avg_pressure_hpa']], 
                                how='left', 
                                left_on='flight_date', 
                                right_on='date').drop(columns = ['date'])


In [ ]:
flights_information_debby = flights_information_debby.merge(weather_sum[['date', 'avg_temp_c', 'precipitation_mm', 'avg_wind_speed_kmh', 'avg_pressure_hpa']], 
                                how='left', 
                                left_on='flight_date', 
                                right_on='date').drop(columns = ['date'])

# Caculate the Correlations #

In [46]:
flights_MCO_dep.head(2)

,flight_date,total_flights,total_cancelled,avg_dep_delay,avg_arr_delay,avg_temp_c_x,precipitation_mm_x,avg_wind_speed_kmh_x,avg_pressure_hpa_x,avg_temp_c_y,precipitation_mm_y,avg_wind_speed_kmh_y,avg_pressure_hpa_y
0,2024-07-01,428,9,15.36,8.25,27.6,0.0,10.1,1015.5,27.6,0.0,10.1,1015.5
1,2024-07-02,422,1,12.80,4.56,28.9,0.0,8.3,1017.0,28.9,0.0,8.3,1017.0


In [32]:
flights_MCO.columns

Index(['flight_date', 'total_flights', 'total_cancelled', 'avg_dep_delay',
       'avg_arr_delay', 'avg_temp_c', 'precipitation_mm', 'avg_wind_speed_kmh',
       'avg_pressure_hpa'],
      dtype='object')

In [33]:
flights_MCO[['total_flights', 'total_cancelled', 'avg_dep_delay',
       'avg_arr_delay', 'avg_temp_c', 'precipitation_mm', 'avg_wind_speed_kmh',
       'avg_pressure_hpa']].corr()

,total_flights,total_cancelled,avg_dep_delay,avg_arr_delay,avg_temp_c,precipitation_mm,avg_wind_speed_kmh,avg_pressure_hpa
total_flights,1.000000,0.189695,0.458424,0.451838,0.049099,0.016036,-0.175949,0.300180
total_cancelled,0.189695,1.000000,0.725840,0.679000,-0.120837,0.173514,0.601542,-0.227933
avg_dep_delay,0.458424,0.725840,1.000000,0.982294,-0.135655,0.262864,0.242488,0.002104
avg_arr_delay,0.451838,0.679000,0.982294,1.000000,-0.183676,0.330356,0.227550,0.011086
avg_temp_c,0.049099,-0.120837,-0.135655,-0.183676,1.000000,-0.276452,-0.008940,-0.181994
precipitation_mm,0.016036,0.173514,0.262864,0.330356,-0.276452,1.000000,0.075715,0.026940
avg_wind_speed_kmh,-0.175949,0.601542,0.242488,0.227550,-0.008940,0.075715,1.000000,-0.650100
avg_pressure_hpa,0.300180,-0.227933,0.002104,0.011086,-0.181994,0.026940,-0.650100,1.000000


In [56]:
flights_TPA[['total_flights', 'total_cancelled', 'avg_dep_delay',
       'avg_arr_delay', 'avg_temp_c', 'precipitation_mm', 'avg_wind_speed_kmh',
       'avg_pressure_hpa']].corr()

,total_flights,total_cancelled,avg_dep_delay,avg_arr_delay,avg_temp_c,precipitation_mm,avg_wind_speed_kmh,avg_pressure_hpa
total_flights,1.000000,0.227294,0.459775,0.507863,-0.201593,0.095747,-0.157563,0.228731
total_cancelled,0.227294,1.000000,0.772814,0.757259,-0.342093,0.517756,0.690282,-0.386768
avg_dep_delay,0.459775,0.772814,1.000000,0.985589,-0.338615,0.474307,0.377264,-0.115577
avg_arr_delay,0.507863,0.757259,0.985589,1.000000,-0.358407,0.472395,0.357658,-0.104570
avg_temp_c,-0.201593,-0.342093,-0.338615,-0.358407,1.000000,-0.569378,-0.009968,-0.186243
precipitation_mm,0.095747,0.517756,0.474307,0.472395,-0.569378,1.000000,0.305104,-0.146008
avg_wind_speed_kmh,-0.157563,0.690282,0.377264,0.357658,-0.009968,0.305104,1.000000,-0.702691
avg_pressure_hpa,0.228731,-0.386768,-0.115577,-0.104570,-0.186243,-0.146008,-0.702691,1.000000


In [57]:
flights_JFK[['total_flights', 'total_cancelled', 'avg_dep_delay',
       'avg_arr_delay', 'avg_temp_c', 'precipitation_mm', 'avg_wind_speed_kmh',
       'avg_pressure_hpa']].corr()

,total_flights,total_cancelled,avg_dep_delay,avg_arr_delay,avg_temp_c,precipitation_mm,avg_wind_speed_kmh,avg_pressure_hpa
total_flights,1.000000,0.062614,0.175096,0.172858,0.148126,-0.206051,0.101449,-0.076593
total_cancelled,0.062614,1.000000,0.776338,0.766440,0.241573,0.660187,0.179976,-0.391817
avg_dep_delay,0.175096,0.776338,1.000000,0.977257,0.203018,0.494702,0.178397,-0.331650
avg_arr_delay,0.172858,0.766440,0.977257,1.000000,0.235010,0.520425,0.235385,-0.361245
avg_temp_c,0.148126,0.241573,0.203018,0.235010,1.000000,0.146809,-0.121281,-0.414049
precipitation_mm,-0.206051,0.660187,0.494702,0.520425,0.146809,1.000000,0.082703,-0.295187
avg_wind_speed_kmh,0.101449,0.179976,0.178397,0.235385,-0.121281,0.082703,1.000000,-0.310653
avg_pressure_hpa,-0.076593,-0.391817,-0.331650,-0.361245,-0.414049,-0.295187,-0.310653,1.000000


In [50]:
flights_MCO_dep[['total_flights', 'total_cancelled', 'avg_dep_delay',
       'avg_arr_delay', 'avg_temp_c', 'precipitation_mm', 'avg_wind_speed_kmh',
       'avg_pressure_hpa']].corr()

,total_flights,total_cancelled,avg_dep_delay,avg_arr_delay,avg_temp_c,precipitation_mm,avg_wind_speed_kmh,avg_pressure_hpa
total_flights,1.000000,0.179552,0.437762,0.423573,0.054170,0.014061,-0.174543,0.295369
total_cancelled,0.179552,1.000000,0.613559,0.560147,-0.125750,0.179043,0.599882,-0.229419
avg_dep_delay,0.437762,0.613559,1.000000,0.967308,-0.180764,0.345691,0.141249,0.079518
avg_arr_delay,0.423573,0.560147,0.967308,1.000000,-0.211104,0.392101,0.130724,0.084389
avg_temp_c,0.054170,-0.125750,-0.180764,-0.211104,1.000000,-0.276452,-0.008940,-0.181994
precipitation_mm,0.014061,0.179043,0.345691,0.392101,-0.276452,1.000000,0.075715,0.026940
avg_wind_speed_kmh,-0.174543,0.599882,0.141249,0.130724,-0.008940,0.075715,1.000000,-0.650100
avg_pressure_hpa,0.295369,-0.229419,0.079518,0.084389,-0.181994,0.026940,-0.650100,1.000000


In [101]:
flights_information_debby[['total_flights', 'total_cancelled', 'avg_dep_delay',
       'avg_arr_delay', 'avg_temp_c', 'precipitation_mm', 'avg_wind_speed_kmh',
       'avg_pressure_hpa']].corr()

,total_flights,total_cancelled,avg_dep_delay,avg_arr_delay,avg_temp_c,precipitation_mm,avg_wind_speed_kmh,avg_pressure_hpa
total_flights,1.000000,0.226726,0.505476,0.526227,0.232923,0.120691,-0.156645,0.169112
total_cancelled,0.226726,1.000000,0.794593,0.764124,0.062508,0.430562,0.569004,-0.395444
avg_dep_delay,0.505476,0.794593,1.000000,0.984073,0.125526,0.327626,0.291504,-0.191774
avg_arr_delay,0.526227,0.764124,0.984073,1.000000,0.127723,0.343913,0.287407,-0.196663
avg_temp_c,0.232923,0.062508,0.125526,0.127723,1.000000,-0.361128,-0.072970,-0.170126
precipitation_mm,0.120691,0.430562,0.327626,0.343913,-0.361128,1.000000,0.232828,-0.054376
avg_wind_speed_kmh,-0.156645,0.569004,0.291504,0.287407,-0.072970,0.232828,1.000000,-0.609636
avg_pressure_hpa,0.169112,-0.395444,-0.191774,-0.196663,-0.170126,-0.054376,-0.609636,1.000000


# Visualisation: Wind Speed #

## Airport MCO ##

In [ ]:
x_mco = flights_MCO['flight_date']
y_mco = flights_MCO['total_cancelled']
y_wind = flights_MCO['avg_wind_speed_kmh']

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x_mco, y=y_mco,
    mode='lines',
    name='Cancellation',
    yaxis='y1'  # Assign to first y-axis
))

fig.add_trace(go.Scatter(
    x=x_mco, y=y_wind,
    mode='lines',
    name='Average Wind Speed (km/h)',
    yaxis='y2'  # Assign to second y-axis
))

fig.update_layout(
    title="Flight Cancellations vs. Wind Speed in MCO",
    xaxis_title="Date",
    yaxis=dict(
        title="Cancellations",
    ),
    yaxis2=dict(
        title="Wind Speed (km/h)",
        overlaying='y',  # Makes it share the same x-axis
        side='right'
    ),
    legend=dict(x=0, y=1),  # Position legend
    font=dict(size=14)  # Sets overall font size
)

fig.show()

In [61]:
x_mco = flights_MCO_dep['flight_date']
y_mco = flights_MCO_dep['avg_dep_delay']
y_wind = flights_MCO_dep['avg_wind_speed_kmh']

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x_mco, y=y_mco,
    mode='lines',
    name='Average Departure Delay (mins)',
    yaxis='y1'  # Assign to first y-axis
))

fig.add_trace(go.Scatter(
    x=x_mco, y=y_wind,
    mode='lines',
    name='Average Wind Speed (km/h)',
    yaxis='y2'  # Assign to second y-axis
))

fig.update_layout(
    title="Average Departure Delay vs. Wind Speed in MCO",
    xaxis_title="Date",
    yaxis=dict(
        title="Average Departure Delay",
    ),
    yaxis2=dict(
        title="Wind Speed (km/h)",
        overlaying='y',  # Makes it share the same x-axis
        side='right'
    ),
    legend=dict(x=0, y=1),  # Position legend
    font=dict(size=14)  # Sets overall font size
)

fig.show()

In [62]:
x_mco = flights_MCO_arr['flight_date']
y_mco = flights_MCO_arr['avg_arr_delay']
y_wind = flights_MCO_arr['avg_wind_speed_kmh']

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x_mco, y=y_mco,
    mode='lines',
    name='Average Arrival Delay (mins)',
    yaxis='y1'  # Assign to first y-axis
))

fig.add_trace(go.Scatter(
    x=x_mco, y=y_wind,
    mode='lines',
    name='Average Wind Speed (km/h)',
    yaxis='y2'  # Assign to second y-axis
))

fig.update_layout(
    title="Average Arrival Delay vs.Wind Speed in MCO",
    xaxis_title="Date",
    yaxis=dict(
        title="Average Arrival Delay",
    ),
    yaxis2=dict(
        title="Wind Speed (km/h)",
        overlaying='y',  # Makes it share the same x-axis
        side='right'
    ),
    legend=dict(x=0, y=1),  # Position legend
    font=dict(size=14)  # Sets overall font size
)

fig.show()

## Airport TPA ## 

In [37]:
x_tpa = flights_TPA['flight_date']
y_tpa = flights_TPA['total_cancelled']
y_wind = flights_TPA['avg_wind_speed_kmh']

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x_tpa, y=y_tpa,
    mode='lines',
    name='Cancellation',
    yaxis='y1'  # Assign to first y-axis
))

fig.add_trace(go.Scatter(
    x=x_tpa, y=y_wind,
    mode='lines',
    name='Average Wind Speed (km/h)',
    yaxis='y2'  # Assign to second y-axis
))

fig.update_layout(
    title="Flight Cancellations vs. Wind Speed in TPA",
    xaxis_title="Date",
    yaxis=dict(
        title="Cancellations",
    ),
    yaxis2=dict(
        title="Wind Speed (km/h)",
        overlaying='y',  # Makes it share the same x-axis
        side='right'
    ),
    legend=dict(x=0, y=1),  # Position legend
    font=dict(size=14)  # Sets overall font size
)

fig.show()

In [ ]:
x_tpa = flights_TPA_dep['flight_date']
y_tpa = flights_TPA_dep['avg_dep_delay']
y_wind = flights_TPA_dep['avg_wind_speed_kmh']

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x_tpa, y=y_tpa,
    mode='lines',
    name='Average Departure Delay (mins)',
    yaxis='y1'  # Assign to first y-axis
))

fig.add_trace(go.Scatter(
    x=x_tpa, y=y_wind,
    mode='lines',
    name='Average Wind Speed (km/h)',
    yaxis='y2'  # Assign to second y-axis
))

fig.update_layout(
    title="Average Departure Delay vs. Wind Speed in TPA",
    xaxis_title="Date",
    yaxis=dict(
        title="Average Departure Delay",
    ),
    yaxis2=dict(
        title="Wind Speed (km/h)",
        overlaying='y',  # Makes it share the same x-axis
        side='right'
    ),
    legend=dict(x=0, y=1),  # Position legend
    font=dict(size=14)  # Sets overall font size
)

fig.show()

In [ ]:
x_tpa = flights_TPA_arr['flight_date']
y_tpa = flights_TPA_arr['avg_arr_delay']
y_wind = flights_TPA_arr['avg_wind_speed_kmh']

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x_tpa, y=y_tpa,
    mode='lines',
    name='Average Arrival Delay (mins)',
    yaxis='y1'  # Assign to first y-axis
))

fig.add_trace(go.Scatter(
    x=x_tpa, y=y_wind,
    mode='lines',
    name='Average Wind Speed (km/h)',
    yaxis='y2'  # Assign to second y-axis
))

fig.update_layout(
    title="Average Arrival Delay vs. Wind Speed in TPA",
    xaxis_title="Date",
    yaxis=dict(
        title="Average Arrival Delay",
    ),
    yaxis2=dict(
        title="Wind Speed (km/h)",
        overlaying='y',  # Makes it share the same x-axis
        side='right'
    ),
    legend=dict(x=0, y=1),  # Position legend
    font=dict(size=14)  # Sets overall font size
)

fig.show()

## Airport JFK ##

In [100]:
x_jfk = flights_JFK['flight_date']
y_jfk = flights_JFK['total_cancelled']
y_wind = flights_JFK['avg_wind_speed_kmh']

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x_jfk, y=y_jfk,
    mode='lines',
    name='Cancellation',
    yaxis='y1'  # Assign to first y-axis
))

fig.add_trace(go.Scatter(
    x=x_jfk, y=y_wind,
    mode='lines',
    name='Average Wind Speed (km/h)',
    yaxis='y2'  # Assign to second y-axis
))

fig.update_layout(
    title="Flight Cancellations vs. Wind Speed in JFK",
    xaxis_title="Date",
    yaxis=dict(
        title="Cancellations",
    ),
    yaxis2=dict(
        title="Wind Speed (km/h)",
        overlaying='y',  # Makes it share the same x-axis
        side='right'
    ),
    legend=dict(x=0, y=1),  # Position legend
    font=dict(size=14)  # Sets overall font size
)

fig.show()

In [99]:
x_jfk = flights_JFK_dep['flight_date']
y_jfk = flights_JFK_dep['avg_dep_delay']
y_wind = flights_JFK_dep['avg_wind_speed_kmh']

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x_jfk, y=y_jfk,
    mode='lines',
    name='Average Departure Delay (mins)',
    yaxis='y1'  # Assign to first y-axis
))

fig.add_trace(go.Scatter(
    x=x_jfk, y=y_wind,
    mode='lines',
    name='Average Wind Speed (km/h)',
    yaxis='y2'  # Assign to second y-axis
))

fig.update_layout(
    title="Average Departure Delay vs. Wind Speed in JFK",
    xaxis_title="Date",
    yaxis=dict(
        title="Average Departure Delay",
    ),
    yaxis2=dict(
        title="Wind Speed (km/h)",
        overlaying='y',  # Makes it share the same x-axis
        side='right'
    ),
    legend=dict(x=0, y=1),  # Position legend
    font=dict(size=14)  # Sets overall font size
)

fig.show()

In [98]:
x_jfk = flights_JFK_arr['flight_date']
y_jfk = flights_JFK_arr['avg_arr_delay']
y_wind = flights_JFK_arr['avg_wind_speed_kmh']

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x_jfk, y=y_jfk,
    mode='lines',
    name='Average Arrival Delay (mins)',
    yaxis='y1'  # Assign to first y-axis
))

fig.add_trace(go.Scatter(
    x=x_jfk, y=y_wind,
    mode='lines',
    name='Average Wind Speed (km/h)',
    yaxis='y2'  # Assign to second y-axis
))

fig.update_layout(
    title="Average Arrival Delay vs. Wind Speed in JFK",
    xaxis_title="Date",
    yaxis=dict(
        title="Average Arrival Delay",
    ),
    yaxis2=dict(
        title="Wind Speed (km/h)",
        overlaying='y',  # Makes it share the same x-axis
        side='right'
    ),
    legend=dict(x=0, y=1),  # Position legend
    font=dict(size=14)  # Sets overall font size
)

fig.show()

# Visualisation: Precipitation #

## Airport MCO ##

In [94]:
x_mco = flights_MCO['flight_date']
y_mco = flights_MCO['total_cancelled']
y_rain = flights_MCO['precipitation_mm']

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x_mco, y=y_mco,
    mode='lines',
    name='Cancellation',
    yaxis='y1'  # Assign to first y-axis
))

fig.add_trace(go.Scatter(
    x=x_mco, y=y_rain,
    mode='lines',
    name='Precipitation (mm)',
    yaxis='y2'  # Assign to second y-axis
))

fig.update_layout(
    title="Flight Cancellations vs. Precipitation in MCO",
    xaxis_title="Date",
    yaxis=dict(
        title="Cancellations",
    ),
    yaxis2=dict(
        title="Precipitation (mm)",
        overlaying='y',  # Makes it share the same x-axis
        side='right'
    ),
    legend=dict(x=0, y=1),  # Position legend
    font=dict(size=14)  # Sets overall font size
)

fig.show()

## Airport TPA ##

In [ ]:
x_tpa = flights_TPA['flight_date']
y_tpa = flights_TPA['total_cancelled']
y_rain = flights_TPA['precipitation_mm']

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x_tpa, y=y_tpa,
    mode='lines',
    name='Cancellation',
    yaxis='y1'  # Assign to first y-axis
))

fig.add_trace(go.Scatter(
    x=x_tpa, y=y_rain,
    mode='lines',
    name='Precipitation (mm)',
    yaxis='y2'  # Assign to second y-axis
))

fig.update_layout(
    title="Flight Cancellations vs. Precipitation in TPA",
    xaxis_title="Date",
    yaxis=dict(
        title="Cancellations",
    ),
    yaxis2=dict(
        title="Precipitation (mm)",
        overlaying='y',  # Makes it share the same x-axis
        side='right'
    ),
    legend=dict(x=0, y=1),  # Position legend
    font=dict(size=14)  # Sets overall font size
)

fig.show()

## Airport JFK ##

In [97]:
x_jfk = flights_JFK['flight_date']
y_jfk = flights_JFK['total_cancelled']
y_rain = flights_JFK['precipitation_mm']

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x_jfk, y=y_jfk,
    mode='lines',
    name='Cancellation',
    yaxis='y1'  # Assign to first y-axis
))

fig.add_trace(go.Scatter(
    x=x_jfk, y=y_rain,
    mode='lines',
    name='Precipitation (mm)',
    yaxis='y2'  # Assign to second y-axis
))

fig.update_layout(
    title="Flight Cancellations vs. Precipitation in JFK",
    xaxis_title="Date",
    yaxis=dict(
        title="Cancellations",
    ),
    yaxis2=dict(
        title="Precipitation (mm)",
        overlaying='y',  # Makes it share the same x-axis
        side='right'
    ),
    legend=dict(x=0, y=1),  # Position legend
    font=dict(size=14)  # Sets overall font size
)

fig.show()

# Visualisation Summary #

In [95]:
x = flights_information_debby['flight_date']
y = flights_information_debby['avg_arr_delay']
y_wind = flights_information_debby['avg_wind_speed_kmh']

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x, y=y,
    mode='lines',
    name='Average Arrival Delay (mins)',
    yaxis='y1'  # Assign to first y-axis
))

fig.add_trace(go.Scatter(
    x=x, y=y_wind,
    mode='lines',
    name='Average Wind Speed (km/h)',
    yaxis='y2'  # Assign to second y-axis
))

fig.update_layout(
    title="Average Arrival Delay vs. Wind Speed",
    xaxis_title="Date",
    yaxis=dict(
        title="Average Arrival Delay",
    ),
    yaxis2=dict(
        title="Wind Speed (km/h)",
        overlaying='y',  # Makes it share the same x-axis
        side='right'
    ),
    legend=dict(x=0, y=1),  # Position legend
    font=dict(size=14)  # Sets overall font size
)

fig.show()

In [91]:
x = flights_information_debby['flight_date']
y = flights_information_debby['total_cancelled']
y_rain = flights_information_debby['precipitation_mm']

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x, y=y,
    mode='lines',
    name='Cancellation',
    yaxis='y1'  # Assign to first y-axis
))

fig.add_trace(go.Scatter(
    x=x, y=y_rain,
    mode='lines',
    name='Precipitation (mm)',
    yaxis='y2'  # Assign to second y-axis
))

fig.update_layout(
    title="Flight Cancellations vs. Precipitation",
    xaxis_title="Date",
    yaxis=dict(
        title="Cancellations",
    ),
    yaxis2=dict(
        title="Precipitation (mm)",
        overlaying='y',  # Makes it share the same x-axis
        side='right'
    ),
    legend=dict(x=0, y=1),  # Position legend
    font=dict(size=14)  # Sets overall font size
)

fig.show()